In [218]:
import aiohttp
import asyncio
import numpy as np
import pickle
import re
import typing as t
import nltk
from nltk.tokenize import word_tokenize
from pymorphy3 import MorphAnalyzer

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\podov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\podov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Test

In [91]:
answers = {}

async def fetch(session: aiohttp.ClientSession, url: str, id):
    async with session.get(url) as response:
        await asyncio.sleep((1 + np.random.sample()) * np.random.randint(1, 4))
        answers[id] = await response.json()


async def main():
    semaphore = asyncio.Semaphore(7)
    
    async def sem_task(task):
        async with semaphore:
            return await task
    
    async with aiohttp.ClientSession() as session:
        tasks = []
        ids = np.arange(2)
        np.random.shuffle(ids)
        for id in ids:
            tasks.append(
                sem_task(fetch(session, f'http://127.0.0.1:10005/test/{id}', id))
            )
        await asyncio.gather(*tasks, return_exceptions=False)

In [92]:
await main()

In [198]:
answers

{0: {'detail': 'Too Many Requests'}, 1: {'detail': 'Too Many Requests'}}

## Not test

In [130]:
async def sema(n_cor: int, tasks: list) -> None:
    semaphore = asyncio.Semaphore(n_cor)

    async def sem_task(task):
        async with semaphore:
            return await task

    tasks = [sem_task(task) for task in tasks]
    await asyncio.gather(*tasks)

In [161]:
res = {}

In [162]:
async def fetch(session: aiohttp.ClientSession, url: str, payload: dict, storage, id):
    async with session.get(url, params=payload) as response:
        await asyncio.sleep(1 + np.random.sample())
        storage[id] = await response.json()

In [163]:
async with aiohttp.ClientSession() as session:
    tasks = []
    for i in range(20):
        url = f'https://api.hh.ru/vacancies'
        payload = {
            'page': i,
            'per_page': 100,
            'text': 'python',
            'area': 113,  # Россия
            'currency': 'RUR',  # Рубль
            'only_with_salary': 'true',
        }
        tasks.append(fetch(session, url, payload, res, i))
    await sema(7, tasks)

In [164]:
res.keys()

dict_keys([0, 3, 6, 1, 2, 5, 4, 7, 10, 8, 9, 12, 11, 13, 14, 15, 16, 17, 18, 19])

In [165]:
for k in res:
    print(res[k]['per_page'], res[k]['pages'], res[k]['page'])

100 20 0
100 20 3
100 20 6
100 20 1
100 20 2
100 20 5
100 20 4
100 20 7
100 20 10
100 20 8
100 20 9
100 20 12
100 20 11
100 20 13
100 20 14
100 20 15
100 20 16
100 20 17
100 20 18
100 20 19


max = 7 запросов / сек

p.s. и уменьшается под конец?

In [166]:
ids = {
    vacancy['id']
    for page in res.values()
    for vacancy in page['items']
}
len(ids)

2000

In [167]:
ids

{'87749899',
 '87375438',
 '86827006',
 '87281935',
 '88043112',
 '87800849',
 '46236237',
 '87415398',
 '86771770',
 '69558404',
 '86687911',
 '87844863',
 '87733672',
 '87291701',
 '86882242',
 '81726419',
 '87981747',
 '87820115',
 '88024584',
 '83078030',
 '87011395',
 '82310598',
 '87613125',
 '87514811',
 '87831521',
 '84399422',
 '87234774',
 '87841519',
 '77114173',
 '87255803',
 '87899823',
 '87624911',
 '73313510',
 '87607335',
 '85990200',
 '87934803',
 '86330756',
 '87168696',
 '87766146',
 '86571284',
 '85725045',
 '87669349',
 '87907521',
 '87679572',
 '87522840',
 '87448622',
 '86631667',
 '87419095',
 '87967222',
 '87230775',
 '87657510',
 '86931980',
 '84496185',
 '87557768',
 '87972467',
 '87426344',
 '86410879',
 '87481366',
 '86336362',
 '86602786',
 '80433705',
 '86857413',
 '86330827',
 '87150635',
 '88005156',
 '87820054',
 '87614164',
 '87406300',
 '87725271',
 '83406279',
 '86872174',
 '87817007',
 '83965537',
 '86241759',
 '86631893',
 '88042953',
 '87548665',

In [168]:
vacancies = {}

In [181]:
ids_to_fetch = ids.copy()
async with aiohttp.ClientSession() as session:
    while len(ids_to_fetch):
        ok_ids = {k for k, v in vacancies.items() if 'id' in v.keys()}
        ids_to_fetch -= ok_ids
        print(f'Ok is {len(ok_ids)}.\t To fetch is {len(ids_to_fetch)}')

        tasks = []
        for id in ids_to_fetch:
            url = f'https://api.hh.ru/vacancies/{id}'
            payload = {}
            tasks.append(fetch(session, url, payload, vacancies, id))
        await sema(7, tasks)

Ok is 2000.	 To fetch is 0


5 + 7 + 4 + 2 = <18

In [200]:
list(vacancies.values())[0]

{'id': '87994663',
 'premium': False,
 'billing_type': {'id': 'standard', 'name': 'Стандарт'},
 'relations': [],
 'name': 'Data Engineer',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': {'from': 2000, 'to': 4000, 'currency': 'USD', 'gross': False},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': None,
 'allow_messages': True,
 'experience': {'id': 'between3And6', 'name': 'От 3 до 6 лет'},
 'schedule': {'id': 'remote', 'name': 'Удаленная работа'},
 'employment': {'id': 'full', 'name': 'Полная занятость'},
 'department': None,
 'contacts': None,
 'description': '<p><strong>Внимание, кандидатов с опытом работы менее 3-х лет просьба не оставлять откликов!</strong></p> <p><strong>Чем предстоит заниматься:</strong></p> <ul> <li>Продуктивизация и поддержка моделей данных;</li> <li>Создание пайплайнов обработки данных: собирать датасеты из существующих источников данных, запускать м

In [184]:
all(x['description'] for x in vacancies.values())

True

In [189]:
path = '../storage/vacancies_2k_.pkl'

# with open(path, 'wb') as f:
#     pickle.dump(vacancies, f)

# with open(path, 'rb') as f:
#     vacancies = pickle.load(f)

In [213]:
corpus = [x['description'] for x in vacancies.values()]

In [214]:
corpus[:5]

['<p><strong>Внимание, кандидатов с опытом работы менее 3-х лет просьба не оставлять откликов!</strong></p> <p><strong>Чем предстоит заниматься:</strong></p> <ul> <li>Продуктивизация и поддержка моделей данных;</li> <li>Создание пайплайнов обработки данных: собирать датасеты из существующих источников данных, запускать модели и доставлять их результат до конечного бизнес-заказчика;</li> <li>Добавление в пайплайн data quality мониторинга и увеличение производительности существующих моделей;</li> <li>Построение архитектуры;</li> </ul> <p><strong>Главные требования:</strong></p> <ul> <li>Опыт реализации, масштабирования, поддержки, интеграции высокогонагруженных сервисов, внутри которых жили построенные Data Scientist’ами алгоритмы и ML-модели;</li> <li>Знание концепций баз данных и программирования;</li> <li>Структурное мышление и умение излагать свои мысли;</li> <li>Опыт работы в электронной коммерции будет плюсом;</li> </ul> <p><strong>Стек</strong></p> <ol> <li><strong>разработка:</st

In [215]:
def apply_re(pattern: str, reply_on: str, corpus: t.Sequence[str]) -> t.MutableSequence[str]:
    return list(map(lambda x: re.sub(pattern, reply_on, x), corpus))

In [216]:
corpus = apply_re('(<[^>]+>|\s)', ' ', corpus)
corpus = apply_re('[^{а-яА-Яa-zA-Z0-9}]', ' ', corpus)
corpus = apply_re('\s+', ' ', corpus)

corpus[:10]

[' Внимание кандидатов с опытом работы менее 3 х лет просьба не оставлять откликов Чем предстоит заниматься Продуктивизация и поддержка моделей данных Создание пайплайнов обработки данных собирать датасеты из существующих источников данных запускать модели и доставлять их результат до конечного бизнес заказчика Добавление в пайплайн data quality мониторинга и увеличение производительности существующих моделей Построение архитектуры Главные требования Опыт реализации масштабирования поддержки интеграции высокогонагруженных сервисов внутри которых жили построенные Data Scientist ами алгоритмы и ML модели Знание концепций баз данных и программирования Структурное мышление и умение излагать свои мысли Опыт работы в электронной коммерции будет плюсом Стек разработка Python 3 10 GitLab Docker Kubernetes Java typescript php данные PostgreSQL Kafka MySQL отчеты и мониторинги Prometheus Grafana Условия Удаленная работа Большая и крутая команда разработчиков Своевременная выплата заработной плат

In [219]:
morph = MorphAnalyzer()
stop_words = stopwords.words('russian')

corpus = [
    [
        word_norm for word in word_tokenize(doc, language='russian')
        if (word_norm := morph.normal_forms(word)[0]) not in stop_words
    ] for doc in corpus
]

corpus[:5]

[['внимание',
  'кандидат',
  'опыт',
  'работа',
  'менее',
  '3',
  'х',
  'год',
  'просьба',
  'оставлять',
  'отклик',
  'предстоять',
  'заниматься',
  'продуктивизация',
  'поддержка',
  'модель',
  'данные',
  'создание',
  'пайплайн',
  'обработка',
  'данные',
  'собирать',
  'датасет',
  'существующий',
  'источник',
  'данные',
  'запускать',
  'модель',
  'доставлять',
  'результат',
  'конечный',
  'бизнес',
  'заказчик',
  'добавление',
  'пайплайна',
  'data',
  'quality',
  'мониторинг',
  'увеличение',
  'производительность',
  'существующий',
  'модель',
  'построение',
  'архитектура',
  'главный',
  'требование',
  'опыт',
  'реализация',
  'масштабирование',
  'поддержка',
  'интеграция',
  'высокогонагруженный',
  'сервис',
  'внутри',
  'который',
  'жить',
  'построить',
  'data',
  'scientist',
  'ами',
  'алгоритм',
  'ml',
  'модель',
  'знание',
  'концепция',
  'база',
  'данные',
  'программирование',
  'структурный',
  'мышление',
  'умение',
  'излагать

In [220]:
path = '../storage/corpus_vacancies.pkl'

# with open(path, 'wb') as f:
#     pickle.dump(corpus, f)

# with open(path, 'rb') as f:
#     corpus = pickle.load(f)

In [4]:
import pickle
z = [1, 2 ,    3, 4, 5, [1, 2, 2, 1]]
with open('\\\\pns\\data\\a.pkl', 'rb') as f:
    # pickle.dump(z, f)
    print(pickle.load(f))

[1, 2, 3, 4, 5, [1, 2, 2, 1]]
